### Download the following utilities
- http://hgdownload.soe.ucsc.edu/admin/exe/macOSX.x86_64/bigWigToBedGraph
- http://hgdownload.soe.ucsc.edu/admin/exe/macOSX.x86_64/hgWiggle
- bedtools
- pandas (Python package)

### Download the following annotations
- https://public.hoffman2.idre.ucla.edu/ernst/R0RG6/LECIF/mm10.LECIFv1.1.bw
- http://hgdownload.cse.ucsc.edu/goldenpath/mm10/phastCons60way/mm10.60way.phastCons.bw
- http://hgdownload.cse.ucsc.edu/goldenpath/mm10/phastCons60way/mm10.60way.phastCons60wayPlacental.bw
- http://hgdownload.cse.ucsc.edu/goldenpath/mm10/phyloP60way/mm10.60way.phyloP60way.bw
- http://hgdownload.cse.ucsc.edu/goldenpath/mm10/phyloP60way/mm10.60way.phyloP60wayPlacental.bw
- https://drive.google.com/file/d/1TbqDipE2zcw2-FEs5aVWKkQ8zYwjOsQq/view?usp=sharing
- https://github.com/shorvath/MammalianMethylationConsortium/blob/main/Annotations%2C%20Amin%20Haghani/Mammals/Mus_musculus.grcm38.100.HorvathMammalMethylChip40.v1.csv

### Write a bed file with all mm10 coordinates

In [ ]:
import pandas as pd

input_filename = 'Mus_musculus.grcm38.100.HorvathMammalMethylChip40.v1.txt'
df = pd.read_csv(input_filename,usecols=['CGid','seqnames','CGstart','CGend']).dropna()
df = df.sort_values(by=['seqnames','CGstart'])
df['chrom'] = "chr"+df['seqnames'] 
df['CGstart'] = df['CGstart'].astype(int)
df['CGend'] = df['CGend'].astype(int)
df[['chrom','CGstart','CGend','CGid']].to_csv('mm10_coord.bed',sep='\t',header=False,index=False)

### Convert bigWig files to gzipped bed files

In [ ]:
%%bash

./bigWigToBedGraph mm10.LECIFv1.1.bw mm10.LECIF.bed
gzip mm10.LECIF.bed

./bigWigToBedGraph mm10.60way.phastCons.bw mm10.phastCons60way.bed
gzip mm10.phastCons60way.bed

./bigWigToBedGraph mm10.60way.phastCons60wayPlacental.bw mm10.phastCons60wayPlacental.bed
gzip mm10.phastCons60wayPlacental.bed

./bigWigToBedGraph mm10.60way.phyloP60way.bw mm10.phyloP60way.bed
gzip mm10.phyloP60way.bed

./bigWigToBedGraph mm10.60way.phyloP60wayPlacental.bw mm10.phyloP60wayPlacental.bed
gzip mm10.phyloP60wayPlacental.bed

### Map LECIF score

In [ ]:
! bedtools map -a mm10_coord.bed -b mm10.LECIF.bed.gz -c 4 -o mean > mm10_coord.LECIF.bed

### Map sequence constraint scores

In [ ]:
%%bash

for f in phastCons60way phyloP60way phastCons60wayPlacental phyloP60wayPlacental; do
    bedtools map -a mm10_coord.bed -b mm10.$f.bed.gz -c 4 -o mean > mm10_coord.$f.bed
done

### Map ConsHMM states

In [ ]:
! bedtools map -a mm10_coord.bed -b 60_states_segmentation.bed.gz -c 4 -o first > mm10_coord.ConsHMM.bed

### Map ENCODE ChromHMM states

In [ ]:
%%bash
path_prefix=http://hgdownload.soe.ucsc.edu/gbdb/mm10/encode3/chromHmm/
output_filename=mm10_coord.ChromHMM.bed

cp mm10_coord.bed $output_filename
for i in `cat mm10_ChromHMM_filenames.txt`; do
    ./bigBedToBed $path_prefix/$i stdout | bedtools map -a $output_filename -b - -c 4 -o first > tmp.bed
    mv tmp.bed $output_filename
done

### Combine them into a formatted csv file

In [ ]:
import pandas as pd

coord_filename = 'mm10_coord.bed'
bed_filenames = ['mm10_coord.LECIF.bed',
                 'mm10_coord.phastCons60way.bed',
                 'mm10_coord.phastCons60wayPlacental.bed',
                 'mm10_coord.phyloP60way.bed',
                 'mm10_coord.phyloP60wayPlacental.bed',
                 'mm10_coord.ConsHMM.bed',
                 'mm10_coord.ChromHMM.bed']

position_col_names = ['chrom','start','end','probeID']

chromhmm_col_name_prefix = 'ChromHMMState_ENCODE_GorkinEtAl2017bioRxiv_'
chromhmm_col_names = [chromhmm_col_name_prefix+i.strip().replace('.bb','').replace('encode3RenChromHmm','') for i in open('mm10_ChromHMM_filenames.txt')]

names = [position_col_names + n for n in [['LECIFScore_KwonErnst2012NatureCommunications'],
                                          ['PhastConsScore_SiepelEtAl2005GenomeResearch'],
                                          ['PhyloPScore_PollardEtAl2009GenomeResearch'],
                                          ['PhastConsPlacentalScore_SiepelEtAl2005GenomeResearch'],
                                          ['PhyloPPlacentalScore_PollardEtAl2009GenomeResearch'],
                                          ['ConsHMMState_Multiz60way_ArnesonEtAl2020NARGenomBioinform'],
                                          chromhmm_col_names]]

df = pd.read_table(coord_filename,header=None,names=position_col_names)
for i in range(len(bed_filenames)):
    input_df = pd.read_table(bed_filenames[i],header=None,names=names[i])
    df = df.merge(input_df,on=position_col_names,how='left')
    
df.to_csv('mm10_annotErnstLab.csv',index=False)